In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
  
tokenizer = AutoTokenizer.from_pretrained("mrm8488/codebert-base-finetuned-stackoverflow-ner")

model = AutoModelForTokenClassification.from_pretrained("mrm8488/codebert-base-finetuned-stackoverflow-ner")

Downloading:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [20]:
import numpy as np

In [4]:
from transformers import pipeline

In [ ]:
example = """import pandas as pd import seaborn as sns import matplotlib.pyplot as plt import numpy as np"""


In [15]:
# Transform input tokens 
inputs = tokenizer("import WebKit protocol GithubWebViewProtocol: UIViewController!", return_tensors="pt")

# Model apply
outputs = model(**inputs)

In [17]:
loss = outputs.loss

In [18]:
loss

In [21]:
logits = outputs.logits

print(np.argmax(logits.detach().numpy()))

274


In [49]:
new_doc = """
import WebKit

protocol GithubWebViewProtocol: UIViewController {
    func close()
    func loadWebView(with: URLRequest)
}

final class GithubWebView: UIViewController, WKNavigationDelegate, GithubWebViewProtocol {

    // MARK: - Components & vars

    var presenter: GithubWebPresenterProtocol?

    lazy var webView: WKWebView = {
        let webView = WKWebView()
        webView.navigationDelegate = self
        return webView
    }()

    // MARK: - Lifecycle

    override func loadView() {
        view = webView
    }

    override func viewDidLoad() {
        super.viewDidLoad()
        presenter?.viewLoaded()
    }

    // MARK: - GithubWebViewProtocol

    func close() {
        dismiss(animated: true, completion: nil)
    }

    func loadWebView(with request: URLRequest) {
        webView.load(request)
    }

    // MARK: - WKNavigationDelegate

    func webView(
        _: WKWebView,
        decidePolicyFor navigationAction: WKNavigationAction,
        decisionHandler: @escaping (WKNavigationActionPolicy) -> Void)
    {
        presenter?.requestForCallbackURL(request: navigationAction.request)
        decisionHandler(.allow)
    }
}
"""

In [54]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)
example = """import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np"""
ner_results = nlp(new_doc)

In [55]:
for ner in ner_results:
    if ner['entity_group'] == 'Library' and len(ner['word'])>4:
        print(ner['word'])

 webkit
